In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

# missingno: 결측치를 시각화하는 라이브러리입니다.

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [ ]:
# 데이터 경로 설정 

# (baseline 방식)
data_dir = os.getenv('HOME', '.') + '/aiffel/kaggle_kakr_housing/data'
train_path = os.path.join(data_dir, 'train.csv')
test_path = os.path.join(data_dir, 'test.csv')
sub_path = os.path.join(data_dir, 'sample_submission.csv')
#sub도불러오는이유 : 1.형식맞추기 2. index 맞추기 용도(id가 1부터 시작하는 게 아니라 중간부터 시작하는경우)

# 데이터 불러오기
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_sub = pd.read_csv(sub_path)

In [ ]:
# target 분리
y = np.log1p(df_train['price'])
df_train.drop(columns=['price'], inplace=True)

# train/test 합치기(is_train 열을 추가하여, 학습 데이터와 테스트 데이터를 구별할 수 있게 합니다.)
df_train['is_train'] = 1
df_test['is_train'] = 0
data = pd.concat([df_train, df_test], axis=0)

# EDA - 결측치 시각화
msno.matrix(data)
plt.title("Missing Values Visualization")
plt.show()

In [ ]:

# Feature Engineering
# 날짜 정보 처리
if 'date' in data.columns:
    data['date'] = data['date'].apply(lambda x: str(x))
    #혹시 date가 숫자(int)로 저장돼 있다면 **문자열(str)**로 변환 뒤에 다시 int로 변환
    data['year'] = data['date'].str[:4].astype(int)
    data['month'] = data['date'].str[4:6].astype(int)
    # date 열은 더 이상 필요하지 않으므로 삭제합니다.
    data.drop(columns=['date'], inplace=True)

# 리노베이션 여부 및 주택 경과연수(외부 데이터를 쓸수 없으니 최대한 보기좋게 가공)
# 리노베이션 여부를 0 또는 1로 표시
# yr_renovated == 0 → 리노베이션 안 함 → 0
# yr_renovated > 0 → 리노베이션 했음 → 1
# "리노베이션 된 집이 더 비싼가?" 같은 걸 학습할 수 있다
data['renovated'] = (data['yr_renovated'] != 0).astype(int)
# 지어진 연도 그대로 쓰는 것보다, 이렇게 나이로 바꾸는 게 모델이 이해하기 쉬움
data['age'] = data['yr_built'].max() - data['yr_built']
# 이제 정보를 renovated랑 age로 요약했으니, 원본 컬럼은 제거
data.drop(columns=['yr_renovated', 'yr_built'], inplace=True)

# 로그 변환
for col in ['sqft_living', 'sqft_lot', 'sqft_living15', 'sqft_lot15']:
    data[col] = np.log1p(data[col])

# 필요없는 열 제거
if 'id' in data.columns:
    data.drop(columns=['id'], inplace=True)

In [ ]:
# train/test 다시 분리 (학습 데이터(X)와 테스트 데이터(X_test)를 다시 나누고, is_train 열은 삭제)
X = data[data['is_train'] == 1].drop(columns=['is_train'])
X_test = data[data['is_train'] == 0].drop(columns=['is_train'])

# 학습/검증 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 평가 함수 정의
# 내부 모델 학습/튜닝	RMSLE	로그변환된 타겟 기준 성능 측정에 더 적합(모델 학습에서 로그 변환을 쓰기 때문)
# 실제 캐글 제출 평가	RMSE	대회가 요구하는 공식 지표이기 때문(추후에 다시 np.expm1로 돌려줌)
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# 모델 + 파라미터
models = {
    "RandomForest": (RandomForestRegressor(random_state=42), {
        'n_estimators': [100], 'max_depth': [10, None]
        # 실험할 하이퍼파라미터 조합: 2
    }),
    "XGBoost": (XGBRegressor(objective='reg:squarederror', random_state=42), {
        'n_estimators': [100], 'max_depth': [3, 6], 'learning_rate': [0.05, 0.1]
        # 실험할 하이퍼파라미터 조합: 4
    }),
    "LightGBM": (LGBMRegressor(random_state=42), {
        'n_estimators': [100], 'num_leaves': [31, 50], 'learning_rate': [0.05, 0.1]
        # 실험할 하이퍼파라미터 조합: 4
    })
}




In [ ]:
# 성능이 제일 좋은 모델을 찾기 위해 초기화하는 코드
best_model = None

# RMSLE 점수는 작을수록 좋다 → 처음에는 무한대로 설정해서 어떤 모델이든 이보다 작으면 갱신되게 만듦
best_score = float('inf')


# 모델별 학습 및 검증 
# (cv=3: 3겹 교차검증)
# scoring='neg_mean_squared_error': MSE가 작을수록 좋은데, GridSearch는 클수록 좋은 점수로 판단하므로 음수로 바꾼 것
# n_jobs=-1: CPU 전부 사용해서 병렬 처리(병렬 처리를 지원하는 함수(예: 사이킷런의 cross_val_score, GridSearchCV, RandomForestClassifier 등)에 사용)
for name, (model, params) in models.items():
    print(f"\n▶ {name} 모델 학습 중...")
    grid = GridSearchCV(model, params, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
    grid.fit(X_train, y_train)
    pred = grid.predict(X_val)
    # y_val, pred는 로그값
    score = rmsle(np.expm1(y_val), np.expm1(pred))
    print(f"{name} RMSLE: {score:.5f} | Best Params: {grid.best_params_}")

    if score < best_score:
        best_score = score
        best_model = grid.best_estimator_
        best_name = name

# 최종 모델 전체 학습(RMSLE=>RMSE)
best_model.fit(X, y)
test_pred = np.expm1(best_model.predict(X_test))

# 결과 저장
submission = pd.read_csv(sub_path)
submission['price'] = test_pred
filename = f'submission_{best_name}_RMSLE_{best_score:.6f}.csv'
submission.to_csv(filename, index=False)
print(f"\n✅ 최종 결과 저장 완료: {filename}")

In [ ]:
#주요 구성

#EDA 시각화 포함: missingno, seaborn 사용

#피처 엔지니어링: 로그변환, 날짜 처리, 리노베이션 여부, 경과연수 등

#모델 3종: RandomForest, XGBoost, LightGBM 전부 학습 + GridSearchCV 튜닝

#성능 평가: RMSLE 기준으로 비교, 가장 좋은 모델 자동 선택

#제출 파일 생성: submission_{모델명}_RMSLE_{점수}.csv 형태로 저장